# Tracker Per-LS DC Notebook

This notebook is intended to provide tools that enable an effective per lumisection (LS) analysis of runs for data certification. It utilizes the DIALS Python API to fetch the monitoring element histograms, as well as the OMS API to obtain metadata on the run including the trigger rate. These tools offer the option to display the integral of a specified reference run in addition to the run under evaluation. This notebook is intended to be run in SWAN. However, in the case that you wish to run it in a local virtual environment, a `pyproject.toml` is included which specifies all of the basic dependencies.

Note that in order to run the OMS API you will need to have a json file with the client ID (`API_CLIENT_ID`) and secret (`API_CLIENT_SECRET`). For more information on how to obtain these, you can access these [slides](https://indico.cern.ch/event/997758/contributions/4191705/attachments/2173881/3670409/OMS%20CERN%20OpenID%20migration%20-%20update.pdf).

## Setup

In [ ]:
# DIALS API
import cmsdials
from cmsdials.auth.client import AuthClient
from cmsdials.auth.bearer import Credentials
from cmsdials import Dials
from cmsdials.filters import LumisectionHistogram1DFilters, LumisectionHistogram2DFilters

auth = AuthClient()
token = auth.device_auth_flow()
creds = Credentials.from_authclient_token(token)

creds = Credentials.from_creds_file()
dials = Dials(creds)

In [ ]:
!git clone https://github.com/roy-cruz/OMSapi
!pip3 install ./OMSapi

In [ ]:
# OMS API
import json
import os

with open("../clientid.json", "r") as file:
    secrets = json.load(file)

os.environ["API_CLIENT_ID"] = secrets["API_CLIENT_ID"]
os.environ["API_CLIENT_SECRET"] = secrets["API_CLIENT_SECRET"]

import oms

oms_fetch = oms.oms_fetch()

In [ ]:
# Plotly
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import plot

In [ ]:
# Other useful libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import awkward as ak

# Misc.
from typing import List
import numpy.typing as npt

In [ ]:
# Importing a list of the available monitoring elements
from MEs import available_MEs
print("Available MEs: \n")
available_MEs

## Helper Functions
Put here any functions you think would help you in your DC. We have put here some useful functions for plotting and processing of the data.

In [ ]:
def extractdata(queryrslt: cmsdials.clients.h2d.models.PaginatedLumisectionHistogram2DList, rtn_df=False) -> np.ndarray:
    """
    Transforms data fetched from dials into a numpy array and a dataframe (second one optional).
    """
    data_df = pd.DataFrame(queryrslt.dict()["results"])
    data_df.sort_values(["run_number", "ls_number"], inplace=True)

    data_arr = []

    if len(data_df["me"].unique()) == 1:
        data_arr = data_df["data"].to_numpy(dtype=np.ndarray)
        data_arr = np.array([np.array([np.array(x) for x in data_arr])])

    else:
        data_arr = []
        for me in data_df["me"].unique():
            me_arr = data_df[data_df["me"] == me]["data"].to_numpy(dtype=np.ndarray)
            me_arr = np.array([np.array(x) for x in me_arr])
            data_arr.append(me_arr)
        try:
            data_arr = np.array(data_arr)
        except:
            data_arr = ak.Array(data_arr)
    if rtn_df:
        return (data_arr, data_df)
    else:
        return data_arr

    

In [ ]:
def extractdata(queryrslt: cmsdials.clients.h2d.models.PaginatedLumisectionHistogram2DList, rtn_df=False) -> np.ndarray:
    """
    Transforms data fetched from dials into a numpy array and a dataframe (second one optional).
    """
    data_df = pd.DataFrame(queryrslt.dict()["results"])
    data_df.sort_values(["run_number", "ls_number"], inplace=True)

    data_dict = {}
    
    for run in data_df["run_number"].unique():
        for me in data_df[data_df["run_number"]==run]["me"].unique():
            me_arr = data_df[data_df["me"] == me]["data"].to_numpy(dtype=np.ndarray)
            me_arr = np.array([np.array(x) for x in me_arr])
        data_dict[run] = {
            me: me_arr
        }

    if len(data_df["me"].unique()) == 1:
        data_arr = data_df["data"].to_numpy(dtype=np.ndarray)
        data_arr = np.array([np.array([np.array(x) for x in data_arr])])

    else:
        data_arr = []
        for me in data_df["me"].unique():
            me_arr = data_df[data_df["me"] == me]["data"].to_numpy(dtype=np.ndarray)
            me_arr = np.array([np.array(x) for x in me_arr])
            data_arr.append(me_arr)
        try:
            data_arr = np.array(data_arr)
        except:
            data_arr = ak.Array(data_arr)

    if rtn_df:
        return (data_dict, data_df)
    else:
        return data_dict
    

In [ ]:
def normalize_by_trig(datas: np.ndarray, trigger_rates: np.ndarray) -> np.ndarray: # TODO: Switch to awkward arrays entirely for data arrays
    if isinstance(datas, np.ndarray):
        if datas.ndim == 3:  # For 1D histograms
            return datas / trigger_rates[:, np.newaxis]
        elif datas.ndim == 4:  # For 2D histograms
            n = datas.shape[2]
            m = datas.shape[3]
            return datas / np.repeat(trigger_rates[:, np.newaxis], n * m, axis=1).reshape(-1, n, m)
        else:
            raise ValueError("Unsupported number of dimensions in datas array. Expected 3 or 4 dimensions.")
    elif isinstance(datas, ak.Array):
        normalized_datas = []
        for i, data in enumerate(datas):
            if data.ndim == 2:  # 1D histogram
                normalized_data = data / trigger_rates[i]
            elif data.ndim == 3:  # 2D histogram
                normalized_data = data / trigger_rates[i]
            else:
                raise ValueError("Unsupported number of dimensions in datas array. Expected 2 or 3 dimensions within awkward array.")
            normalized_datas.append(normalized_data)
        return ak.Array(normalized_datas)
    else:
        raise ValueError("Unsupported data type. Expected numpy.ndarray or awkward.Array.")

In [ ]:
def multiplotinteractive1D(
        datas: List[np.ndarray], 
        bin_locs: List[np.ndarray], 
        titles: List[str], 
        x_labels: List[str], 
        y_labels: List[str],
        fig_title: str,
        trigger_rates = None,
        normalized_area = True, 
        normalized_trig = False,
        show = True,
        ):
    """
    For plotting multiple 1D histograms
    """
    num_mes = len(titles)
    
    if not (len(datas) == len(bin_locs) == len(titles) == len(x_labels) == len(y_labels)):
        raise ValueError("All input lists must have the same length.")

    num_rows = (num_mes + 1) // 2
    num_cols = 2 if num_mes > 1 else 1
    num_lss = len(datas[0])

    fig = make_subplots(
        rows=num_rows, cols=num_cols, 
        subplot_titles=titles,
        vertical_spacing=0.15,
        horizontal_spacing=0.05
    )

    if normalized_trig:
        if trigger_rates is None:
            raise ValueError("No trigger rate array given.")
        datas = normalize_by_trig(datas, trigger_rates)

    if normalized_area:
        datas = [data / data.sum(axis=1, keepdims=True) for data in datas]

    # Adding the first LS
    for i in range(num_mes):
        row = (i // num_cols) + 1
        col = (i % num_cols) + 1
        fig.add_trace(go.Bar(x=bin_locs[i], y=datas[i][0], name=titles[i]), row=row, col=col)
        fig.update_xaxes(title_text=x_labels[i], row=row, col=col)
        fig.update_yaxes(title_text=y_labels[i], row=row, col=col)

    # Making the steps to update to the rest of the LSs
    steps = []
    for i in range(num_lss):
        step = {
            "method": "restyle",
            "args": [
                {"y": [datas[j][i,:] for j in range(len(datas))]},
                np.arange(len(datas))  # Indices of the traces to modify
            ],
            "label": f"LS {i+1}"
        }
        steps.append(step)
        
    sliders = [{
        "active": 0,
        "currentvalue": {"prefix": 'LS: '},
        "pad": {"t": 50},
        "steps": steps
    }]

    # Adding elements and updating layout
    fig.update_layout(
        sliders=sliders,
        title_text=fig_title,
        bargap=0,
        showlegend=False,
        width=1600,
        height=900
    )

    for i in range(num_mes):
        row = (i // num_cols) + 1
        col = (i % num_cols) + 1
        max_y = datas[i].max()
        fig.update_yaxes(range=[0, max_y], row=row, col=col)

    if show:
        fig.show()
    else:
        return fig

In [ ]:
def multiheatmaps1D(
    datas: List[np.ndarray], 
    bin_locs: List[np.ndarray], 
    titles: List[str], 
    x_labels: List[str], 
    y_labels: List[str],
    fig_title: str,
    trigger_rates = None,
    normalized_trig = False,
    show = True,
    ):

    num_mes = len(titles)

    if not (len(datas) == len(bin_locs) == len(titles) == len(x_labels) == len(y_labels) == num_mes):
        raise ValueError("All input lists must have the same length.")

    num_rows = (num_mes + 1) // 2
    num_cols = 2 if num_mes > 1 else 1
    num_lss = len(datas[0])

    if normalized_trig:
        if trigger_rates is None:
            raise ValueError("No trigger rate array given.")
        datas = normalize_by_trig(datas, trigger_rates)

    fig = make_subplots(
        rows=num_rows, cols=num_cols, 
        subplot_titles = titles,
        vertical_spacing = 0.1,
        horizontal_spacing = 0.1
    )

    for i, data in enumerate(datas):
        row = (i // 2) + 1
        col = (i % 2) + 1
        fig.add_trace(
            go.Heatmap(z=data, x=bin_locs[i], colorscale="Viridis", showscale=False),
            row=row, col=col,
        )
        fig.update_xaxes(title_text=x_labels[i], row=row, col=col)
        fig.update_yaxes(title_text=y_labels[i], row=row, col=col)

    fig.update_layout(
        title_text=fig_title,
        title_font={"size": 24},
        height=1100,
        width=1100,
        annotations = [dict(text = ME, font={"size": 14}, showarrow=False) for ME in titles],
    )

    fig.update_yaxes(autorange="reversed")

    if show:
        fig.show()
    else:
        return fig

In [ ]:
def multiplotinteractive2D(
    datas: List[np.array], 
    x_bin_locs: List[np.array], 
    y_bin_locs: List[np.array],
    titles: List[str], 
    x_labels: List[str], 
    y_labels: List[str],
    fig_title: str,
    trigger_rates = None,
    normalized_trig = False,
    show = True,
    ):
    
    num_mes = len(titles)
    num_rows = (num_mes + 1) // 2
    num_cols = 2 if num_mes > 1 else 1
    num_lss = len(datas[0])

    if normalized_trig:
        if trigger_rates is None:
            raise ValueError("No trigger rate array given.")
        datas = normalize_by_trig(datas, trigger_rates)

    fig = make_subplots(
        rows=num_rows, cols=num_cols, subplot_titles=titles, vertical_spacing = 0.1)
    
    
    for i, data in enumerate(datas):
        fig.add_trace(
            go.Heatmap(
                z = data[i], 
                colorscale = "Viridis",
                zmin = datas.min() if isinstance(datas, np.ndarray) else ak.min(datas),
                zmax = datas.max() if isinstance(datas, np.ndarray) else ak.max(datas)
            ),
            row = i // 2 + 1, col = i % 2 + 1
        )

    # Making the steps to update to the rest of the LSs
    steps = []
    for i in range(num_lss):
        step = dict(
            method = "restyle",
            args = [
                dict(
                    z = [datas[j][i] for j in range(len(datas))],
                ),
                np.arange(len(datas))  # Indices of the traces to modify
            ],
            label = str(i+1)
        )
        steps.append(step)
        
    sliders = [
        dict(
            active = 0,
            currentvalue = dict(prefix = "LS: "),
            pad = dict(t = 50),
            steps = steps
        )
    ]

    # Button
    play_button = dict(
        type = "buttons",
        showactive = False,
        buttons = [
            dict(
                label = "Play",
                method = "animate",
                args = [
                    None, 
                    dict(
                        frame = dict(duration = 500, redraw = True),  # Speed of playback in milliseconds per step
                        fromcurrent = True,
                        transition = dict(duration = 300),
                        mode = "immediate"
                    ),
                ]
            )
        ]
    )

    # Define frames for animation 
    frames = []
    for i in range(num_lss):
        frame = dict(
            data=[
                go.Heatmap(z = datas[j][i]) for j in range(len(datas))
            ],
            name = str(i+1),
            traces = np.arange(len(datas)),
            layout = dict(sliders=[dict(active = i)])  # Update the slider position
        )
        frames.append(frame)

    fig.frames = frames

    fig.update_annotations(font = dict(size = 12))
    fig.update_layout(
        updatemenus = [play_button],
        sliders = sliders,
        bargap = 0,
        showlegend = False,
        width = 1200,
        height = 1000,
    )

    if show:
        fig.show()
    else:
        return fig


## Using OMS to Obtain Metadata

Using the OMS API, we can access important information regarding the run conditions and other information about the run. The available endpoints are:

* `lumisections`
* `runs`
* `fills`
* `datasetrates`

If you wish to access per-LS trigger rate, you can use the last one as shown below.

In [ ]:
runnb = 380237

oms_json = oms_fetch.get_oms_json(api_endpoint="lumisections", runnb=runnb)
oms_df = oms.oms_utils.makeDF(oms_json)
oms_df.columns

In [ ]:
oms_df.head(5)

In [ ]:
# Getting per-LS trigger rates

extrafilter = dict(
    attribute_name="dataset_name",
    value="ZeroBias",
    operator="EQ",
)

attributes = [
    'start_time', 
    'last_lumisection_number', 
    'rate', 
    'run_number',
    'last_lumisection_in_run', 
    'first_lumisection_number', 
    'dataset_name',
    'cms_active', 
    'events'
]

omstrig_json = oms.get_oms_data(
    oms_fetch.omsapi, 
    'datasetrates', 
    runnb,
    extrafilters=[extrafilter]
)

omstrig_df = oms.oms_utils.makeDF(omstrig_json)

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x = omstrig_df["last_lumisection_number"],
        y = omstrig_df["rate"],
    )
)

fig.update_layout(
    title = f"Trigger rate per-LS for Run {runnb}",
    xaxis_title = "Lumisection",
    yaxis_title = "Trigger rate",
)

fig.show()

We store the trigger rate in an array to use later on.

In [ ]:
# Getting trigger rate array
trig_rate = np.array(omstrig_df["rate"])

## 1D Monitoring Elements

In [ ]:
LumisectionHistogram1DFilters?

We fetch data from DIALS as shown here. If you are unfamiliar with regex syntax, you can take a look at the following [cheat sheet](https://www.rexegg.com/regex-quickstart.html) for a quick overview.

In [ ]:
runnb = 380238

data1D = dials.h1d.list_all(
    LumisectionHistogram1DFilters(
        next_token = None,
        dataset_id = None,
        file_id = None,
        run_number = runnb,
        run_number__lte = None,
        run_number__gte = None,
        ls_number = None,
        ls_numbet__lte = None,
        ls_number__gte = None,
        me_id = None,
        entries__gte = None,
        dataset = "/ZeroBias/Run2024C-PromptReco-v1/DQMIO",
        dataset__regex = None,
        logical_file_name = None,
        logical_file_name__regex = None,
        me = None,
        me__regex = "PixelPhase1/Tracks/PXBarrel/charge_PXLayer_."
    ),
    # max_pages=200
)

data1D_arr, data1D_df = extractdata(queryrslt=data1D, rtn_df=True)
MEs_fetched = list(data1D_df["me"].unique())

In [ ]:
# Hitogram bin locations are computed
bin_locs = []
for ME in MEs_fetched:
    x_min = data1D_df[data1D_df["me"] == ME]["x_min"].iloc[0]
    x_max = data1D_df[data1D_df["me"] == ME]["x_max"].iloc[0]
    x_bin = data1D_df[data1D_df["me"] == ME]["x_bin"].iloc[0]
    bin_locs.append(np.linspace(x_min, x_max, int(x_bin)))

In [ ]:
multiplotinteractive1D?

`multiplotinteractive1D` offers the ability to immediately show the plot inside the notebook. However, unless you only intend to make one or two plots, this is not recommended as Plotly plots are resource intensive. It is recommended that you instead do as the following example where we set `show = False` so that the function returns the figure and that you then export the plot as an `html` file which you can then see in your browser.

In [ ]:
x_labels = ["Charge (e)"] * len(MEs_fetched)
y_labels = ["Count"] * len(MEs_fetched)
fig_title = f"Pixel Barrel Charge (Run {runnb})"

fig = multiplotinteractive1D(
    datas = data1D_arr[], 
    bin_locs = bin_locs, 
    titles = MEs_fetched, 
    x_labels = x_labels, 
    y_labels = y_labels,
    trigger_rates = trig_rate,
    fig_title=fig_title,
    normalized_area = False,
    normalized_trig = False,
    show = False
)

# Export plot to html
plot(fig, filename="me_plot.html")

### Heatmaps

By "stacking" 1D histograms, we can create heatmaps which give us an idea of how the run evolved through time as data was being taken.

In [ ]:
multiheatmaps1D?

In [ ]:
hm_xlabels = [""] * len(MEs_fetched)
hm_ylabels = [""] * len(MEs_fetched)
hm_fig_title = f"Pixel Barrel Charge (Run {runnb})"

fig = multiheatmaps1D(
    datas = data1D_arr,
    bin_locs = bin_locs,
    titles = MEs_fetched,
    x_labels = hm_xlabels,
    y_labels = hm_ylabels,
    fig_title = hm_fig_title,
    trigger_rates = trig_rate,
    normalized_trig = True,
    show=False
)

plot(fig, filename="me_plot.html")

## 2D Monitoring Elements

In [ ]:
LumisectionHistogram2DFilters?

2D monitoring elements are also available in DIALS and they can be accessed as shown below.

In [ ]:
runnb = 380238

data2D = dials.h2d.list_all(
    LumisectionHistogram1DFilters(
        next_token = None,
        dataset_id = None,
        file_id = None,
        run_number = runnb,
        run_number__lte = None,
        run_number__gte = None,
        ls_number = None,
        ls_numbet__lte = None,
        ls_number__gte = None,
        me_id = None,
        entries__gte = None,
        dataset = "/ZeroBias/Run2024C-PromptReco-v1/DQMIO",
        dataset__regex = None,
        logical_file_name = None,
        logical_file_name__regex = None,
        me = None,
        me__regex = "PixelPhase1/Phase1_MechanicalView/PXBarrel/digi_occupancy_per_SignedModuleCoord_per_SignedLadderCoord_PXLayer_."
    ),
    # max_pages=200
)

data2D_arr, data2D_df = extractdata(queryrslt=data2D, rtn_df=True)
MEs2D_fetched = list(data2D_df["me"].unique())

In [ ]:
# Computing histogram bin locations in x and y
x_bin_locs = []
y_bin_locs = []
for ME in MEs2D_fetched:
    x_min = data2D_df[data2D_df["me"] == ME]["x_min"].iloc[0]
    x_max = data2D_df[data2D_df["me"] == ME]["x_max"].iloc[0]
    x_bin = data2D_df[data2D_df["me"] == ME]["x_bin"].iloc[0]
    x_bin_locs.append(np.linspace(x_min, x_max, int(x_bin)))

    y_min = data2D_df[data2D_df["me"] == ME]["y_min"].iloc[0]
    y_max = data2D_df[data2D_df["me"] == ME]["y_max"].iloc[0]
    y_bin = data2D_df[data2D_df["me"] == ME]["y_bin"].iloc[0]
    y_bin_locs.append(np.linspace(y_min, y_max, int(y_bin)))

In [ ]:
multiplotinteractive2D?

Like `multiplotinteractive1D`, `multiplotinteractive2D` also provides an option to return the figure instead of immediately plotting. For 2D histograms, this is highly recommended even if you are thinking of only making a single plot, as 2D histograms are particularly resource intensive and can cause issues with the notebook.

In [ ]:
x_labels = ["z"] * len(MEs2D_fetched)
y_labels = ["phi"] * len(MEs2D_fetched)
fig_title = f"Cluster position for PX Barrel (Run {runnb})"

fig2D = multiplotinteractive2D(
    datas = data2D_arr, 
    x_bin_locs = x_bin_locs, 
    y_bin_locs = y_bin_locs,
    titles = MEs2D_fetched, 
    x_labels = x_labels, 
    y_labels = y_labels,
    fig_title = fig_title,
    trigger_rates = trig_rate,
    normalized_trig = True,
    show = False,
)

plot(fig2D, filename="me2D_plot.html")